In [1]:
!pip install langchain chromadb langchain-community faiss-cpu openai tiktoken langchain_openai wikipedia

In [2]:
pip install -U langchain langchain-community langchain-huggingface chromadb

## Wikipedia retriever

In [3]:
from langchain_community.retrievers import WikipediaRetriever

#initialize the retriver, provide the language and set top_k
retriever= WikipediaRetriever(lang="en", top_k_results=2)

query = """Geopolitics of India and Pakistan from Russia's perspective"""

docs = retriever.invoke(query)

docs

[Document(metadata={'title': 'Prisoners of Geography', 'summary': "Prisoners of Geography: Ten Maps That Tell You Everything You Need to Know About Global Politics is a 2015 non-fiction book about geopolitics by the British author and journalist Tim Marshall.\nThe author has also released a children's illustrated version of this book in 2019, Prisoners of Geography - Our World Explained in 12 Simple Maps, nominated for Waterstones Book of the Year. The Power of Geography, a sequel, was released in 2021.", 'source': 'https://en.wikipedia.org/wiki/Prisoners_of_Geography'}, page_content='Prisoners of Geography: Ten Maps That Tell You Everything You Need to Know About Global Politics is a 2015 non-fiction book about geopolitics by the British author and journalist Tim Marshall.\nThe author has also released a children\'s illustrated version of this book in 2019, Prisoners of Geography - Our World Explained in 12 Simple Maps, nominated for Waterstones Book of the Year. The Power of Geograph

In [4]:
## print document
for i,doc in enumerate(docs):
  print(f"\n -- Result{i+1}")
  print(f"Content:\n{doc.page_content}")


 -- Result1
Content:
Prisoners of Geography: Ten Maps That Tell You Everything You Need to Know About Global Politics is a 2015 non-fiction book about geopolitics by the British author and journalist Tim Marshall.
The author has also released a children's illustrated version of this book in 2019, Prisoners of Geography - Our World Explained in 12 Simple Maps, nominated for Waterstones Book of the Year. The Power of Geography, a sequel, was released in 2021.


== Summary ==
Prisoners of Geography is a set of observations about past and present geopolitics seen through the lens of geography. Through various global examples, Tim Marshall challenges the widely held belief that technology is allowing humans to overcome the constraints and vulnerabilities imposed by geography and to render it irrelevant to geopolitics and conflicts. Marshall makes the case for putting the ‘geo’ back in ‘geopolitics’, arguing that geography often gets written out of discussions and analyses of historic and c

## Vector store retriever

In [ ]:
import os
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "***"


embedding_model = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2"
)

In [6]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="LangChain is a framework that helps developers build applications powered by large language models.",
        metadata={
            "source": "langchain_overview",
            "topic": "introduction",
            "doc_id": 1
        }
    ),
    Document(
        page_content="LangChain provides abstractions for prompts, chains, agents, retrievers, and vector stores.",
        metadata={
            "source": "langchain_components",
            "topic": "architecture",
            "doc_id": 2
        }
    ),
    Document(
        page_content="Developers use LangChain to connect LLMs with external tools, APIs, and data sources.",
        metadata={
            "source": "langchain_use_cases",
            "topic": "usage",
            "doc_id": 3
        }
    ),
    Document(
        page_content="LangChain supports both open-source and proprietary language models such as OpenAI, Hugging Face, and Anthropic.",
        metadata={
            "source": "langchain_models",
            "topic": "models",
            "doc_id": 4
        }
    ),
    Document(
        page_content="LangChain is commonly used for building chatbots, retrieval-augmented generation systems, and AI agents.",
        metadata={
            "source": "langchain_applications",
            "topic": "applications",
            "doc_id": 5
        }
    )
]


vector_store = Chroma.from_documents(
    documents,
    embedding= embedding_model,
    collection_name = "my_collection"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
## make Vector store as a retriever
retriever_vector = vector_store.as_retriever(search_kwargs={"k": 2})

## query
query = """what is langchain?"""
result = retriever_vector.invoke(query)
result


[Document(metadata={'source': 'langchain_overview', 'doc_id': 1, 'topic': 'introduction'}, page_content='LangChain is a framework that helps developers build applications powered by large language models.'),
 Document(metadata={'source': 'langchain_applications', 'doc_id': 5, 'topic': 'applications'}, page_content='LangChain is commonly used for building chatbots, retrieval-augmented generation systems, and AI agents.')]

In [8]:
for i,doc in enumerate(result):
  print(f"\n -- Result{i+1} --")
  print(f"Content:\n{doc.page_content}")
  print(f"Source:\n{doc.metadata['source']}")


 -- Result1 --
Content:
LangChain is a framework that helps developers build applications powered by large language models.
Source:
langchain_overview

 -- Result2 --
Content:
LangChain is commonly used for building chatbots, retrieval-augmented generation systems, and AI agents.
Source:
langchain_applications


In [9]:
result_vector_store_direct = vector_store.similarity_search(query,k=2)
for i,doc in enumerate(result_vector_store_direct):
  print(f"\n -- Result{i+1} --")
  print(f"Content:\n{doc.page_content}")
  print(f"Source:{doc.metadata['source']}")


 -- Result1 --
Content:
LangChain is a framework that helps developers build applications powered by large language models.
Source:langchain_overview

 -- Result2 --
Content:
LangChain is commonly used for building chatbots, retrieval-augmented generation systems, and AI agents.
Source:langchain_applications


## Maximal Marginal Relevance(MMR)

In [10]:
documents = [
    Document(
        page_content="LangChain is a developer framework for building LLM-powered applications and workflows.",
        metadata={"source": "langchain_overview_v2", "topic": "introduction", "doc_id": 101}
    ),
    Document(
        page_content="LangChain helps teams create apps that combine large language models with custom logic and data.",
        metadata={"source": "langchain_overview_v3", "topic": "introduction", "doc_id": 102}
    ),
    Document(
        page_content="LangChain offers building blocks like prompt templates, chains, agents, retrievers, and vector databases.",
        metadata={"source": "langchain_components_v2", "topic": "architecture", "doc_id": 201}
    ),
    Document(
        page_content="With LangChain, you can compose prompts, runnables, retrievers, and vector stores to build end-to-end pipelines.",
        metadata={"source": "langchain_components_v3", "topic": "architecture", "doc_id": 202}
    ),
    Document(
        page_content="Many developers use LangChain to connect LLMs to tools, APIs, and private datasets for real applications.",
        metadata={"source": "langchain_use_cases_v2", "topic": "usage", "doc_id": 301}
    ),
    Document(
        page_content="LangChain is often used to wire language models with external services and data sources using tool calls.",
        metadata={"source": "langchain_use_cases_v3", "topic": "usage", "doc_id": 302}
    ),
    Document(
        page_content="LangChain integrates with both open-source and hosted LLMs, including OpenAI, Hugging Face, and Anthropic models.",
        metadata={"source": "langchain_models_v2", "topic": "models", "doc_id": 401}
    ),
    Document(
        page_content="You can use LangChain with proprietary providers or open models via Hugging Face, depending on your stack.",
        metadata={"source": "langchain_models_v3", "topic": "models", "doc_id": 402}
    ),
    Document(
        page_content="Common LangChain applications include chatbots, RAG (retrieval-augmented generation), and autonomous agents.",
        metadata={"source": "langchain_applications_v2", "topic": "applications", "doc_id": 501}
    ),
    Document(
        page_content="Teams frequently adopt LangChain for chat interfaces, RAG systems, and agentic workflows that use tools.",
        metadata={"source": "langchain_applications_v3", "topic": "applications", "doc_id": 502}
    ),
]


In [11]:
from langchain_community.vectorstores import FAISS

## model embedding initiation
embedding_model = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2"
)

##creating vector store from the document
vector_store_faiss = FAISS.from_documents(
    documents,
    embedding_model
)

##Enable Vector store as a retriever
retriever_mmr = vector_store_faiss.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k": 3, "lambda_mult": 0.5} ## k = number of results, lambda_mult = relevance-diversity balance
)

##user query
query = "what is langchain?"
result_mmr = retriever_mmr.invoke(query)



In [12]:
for i,doc in enumerate(result_mmr): ##for 1
  print(f"\n -- Result{i+1} --")
  print(f"Content:\n{doc.page_content}")
  print(f"Source:\n{doc.metadata['source']}")


 -- Result1 --
Content:
LangChain is often used to wire language models with external services and data sources using tool calls.
Source:
langchain_use_cases_v3

 -- Result2 --
Content:
LangChain is a developer framework for building LLM-powered applications and workflows.
Source:
langchain_overview_v2

 -- Result3 --
Content:
Common LangChain applications include chatbots, RAG (retrieval-augmented generation), and autonomous agents.
Source:
langchain_applications_v2


In [13]:
for i,doc in enumerate(result_mmr): ##for 0.5
  print(f"\n -- Result{i+1} --")
  print(f"Content:\n{doc.page_content}")
  print(f"Source:\n{doc.metadata['source']}")


 -- Result1 --
Content:
LangChain is often used to wire language models with external services and data sources using tool calls.
Source:
langchain_use_cases_v3

 -- Result2 --
Content:
LangChain is a developer framework for building LLM-powered applications and workflows.
Source:
langchain_overview_v2

 -- Result3 --
Content:
Common LangChain applications include chatbots, RAG (retrieval-augmented generation), and autonomous agents.
Source:
langchain_applications_v2


## MultiQuery retrievers

In [14]:
# !pip uninstall langchain -y
!pip install -U langchain langchain-community

In [15]:
import langchain, inspect
print("version:", langchain.__version__)
print("file:", langchain.__file__)
print("has retrievers?", hasattr(langchain, "retrievers"))


version: 1.2.0
file: /usr/local/lib/python3.12/dist-packages/langchain/__init__.py
has retrievers? False


In [16]:
import os, glob
print([p for p in glob.glob("langchain*")])


[]


In [17]:
pip install -U langchain-classic

In [18]:
pip -q install -U langchain langchain-community langchain-classic chromadb transformers sentencepiece

In [ ]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_huggingface import HuggingFaceEndpointEmbeddings,ChatHuggingFace,HuggingFaceEndpoint
import os
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "**"


embedding_model = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2"
)

embedding_model = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2"
)

In [20]:
from langchain_core.documents import Document

documents = [

    # ---------------- Diabetes : Biguanides ----------------
    Document(
        page_content=(
            "Metformin is a first-line oral medication for type 2 diabetes that lowers blood sugar "
            "by improving insulin sensitivity and reducing glucose production in the liver."
        ),
        metadata={"disease": "diabetes", "brand": "Glycomet", "class": "biguanide", "doc_id": 1}
    ),
    Document(
        page_content=(
            "Glycomet is commonly prescribed in type 2 diabetes to control fasting glucose levels "
            "by decreasing hepatic glucose output."
        ),
        metadata={"disease": "diabetes", "brand": "Glycomet", "class": "biguanide", "doc_id": 2}
    ),

    # ---------------- Diabetes : Sulfonylureas ----------------
    Document(
        page_content=(
            "Amaryl contains glimepiride, a sulfonylurea that helps manage diabetes by stimulating "
            "the pancreas to release more insulin."
        ),
        metadata={"disease": "diabetes", "brand": "Amaryl", "class": "sulfonylurea", "doc_id": 3}
    ),
    Document(
        page_content=(
            "Glimepiride-based medicines like Amaryl are often used when metformin alone is not sufficient "
            "to achieve glycemic control."
        ),
        metadata={"disease": "diabetes", "brand": "Amaryl", "class": "sulfonylurea", "doc_id": 4}
    ),

    # ---------------- Diabetes : DPP-4 Inhibitors ----------------
    Document(
        page_content=(
            "Januvia is a DPP-4 inhibitor that improves blood sugar control in adults with type 2 diabetes "
            "by increasing incretin hormone levels."
        ),
        metadata={"disease": "diabetes", "brand": "Januvia", "class": "DPP-4 inhibitor", "doc_id": 5}
    ),
    Document(
        page_content=(
            "DPP-4 inhibitors like Januvia help regulate post-meal glucose levels without causing significant hypoglycemia."
        ),
        metadata={"disease": "diabetes", "brand": "Januvia", "class": "DPP-4 inhibitor", "doc_id": 6}
    ),

    # ---------------- Diabetes : Lifestyle & Prevention ----------------
    Document(
        page_content=(
            "Lifestyle changes such as regular exercise, weight management, and a balanced diet "
            "play a critical role in preventing type 2 diabetes."
        ),
        metadata={"disease": "diabetes", "category": "prevention", "doc_id": 7}
    ),
    Document(
        page_content=(
            "Reducing refined carbohydrates, increasing fiber intake, and maintaining physical activity "
            "can significantly lower the risk of developing diabetes."
        ),
        metadata={"disease": "diabetes", "category": "prevention", "doc_id": 8}
    ),

    # ---------------- Diabetes : General Management ----------------
    Document(
        page_content=(
            "Long-term diabetes management focuses on maintaining optimal blood glucose levels "
            "to prevent complications such as neuropathy, nephropathy, and cardiovascular disease."
        ),
        metadata={"disease": "diabetes", "category": "management", "doc_id": 9}
    ),
    Document(
        page_content=(
            "Effective diabetes care often involves a combination of medication, lifestyle modification, "
            "and regular monitoring of blood sugar levels."
        ),
        metadata={"disease": "diabetes", "category": "management", "doc_id": 10}
    ),
]


In [21]:
import shutil, os

vector_store = Chroma.from_documents(
    documents,
    embedding_model,
    collection_name = "diseases",
    persist_directory = "./chroma_db"
)

In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

model_id = "google/flan-t5-base"  # base is faster in colab; large can be slow
tok = AutoTokenizer.from_pretrained(model_id)
mdl = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=mdl,
    tokenizer=tok,
    max_new_tokens=128,
    do_sample=False,
)

llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu
/tmp/ipython-input-3654955188.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [23]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever

retriever_multi_query = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_kwargs={"k": 5}),
    llm=llm,
)

retriever_basic = vector_store.as_retriever(search_kwargs={"k": 5})

In [24]:
query = "How do I stay away from diabetes?"

result_basic = retriever_basic.invoke(query)
result_multi_query = retriever_multi_query.invoke(query)

print("\n===== BASIC SIMILARITY RETRIEVER =====")
for i, doc in enumerate(result_basic, start=1):
    print(f"\n-- Result {i} --")
    print("Content:")
    print(doc.page_content)
    print("Source:")
    print(doc.metadata.get("source"))

print("\n" + "*" * 150)

print("\n===== MULTI QUERY RETRIEVER =====")
for i, doc in enumerate(result_multi_query, start=1):
    print(f"\n-- Result {i} --")
    print("Content:")
    print(doc.page_content)
    print("Source:")
    print(doc.metadata.get("source"))



===== BASIC SIMILARITY RETRIEVER =====

-- Result 1 --
Content:
Long-term diabetes management focuses on maintaining optimal blood glucose levels to prevent complications such as neuropathy, nephropathy, and cardiovascular disease.
Source:
None

-- Result 2 --
Content:
Effective diabetes care often involves a combination of medication, lifestyle modification, and regular monitoring of blood sugar levels.
Source:
None

-- Result 3 --
Content:
Reducing refined carbohydrates, increasing fiber intake, and maintaining physical activity can significantly lower the risk of developing diabetes.
Source:
None

-- Result 4 --
Content:
Lifestyle changes such as regular exercise, weight management, and a balanced diet play a critical role in preventing type 2 diabetes.
Source:
None

-- Result 5 --
Content:
Glycomet is commonly prescribed in type 2 diabetes to control fasting glucose levels by decreasing hepatic glucose output.
Source:
None

*********************************************************

## Contextual Compression retrievr

In [27]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEndpointEmbeddings, HuggingFaceEndpoint
from langchain_core.documents import Document

from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor


In [29]:
from langchain_core.documents import Document

documents = [

    # ================= Newton's Laws of Motion =================
    Document(
        page_content=(
            "Newton's first law of motion, often called the law of inertia, states that an object will "
            "remain at rest or in uniform motion unless an external force acts upon it. This explains "
            "why objects resist changes in their state of motion."
        ),
        metadata={"domain": "physics", "law": "motion", "scientist": "Newton", "doc_id": 1}
    ),
    Document(
        page_content=(
            "The principle of inertia describes how bodies maintain their velocity unless influenced "
            "by an external force. Newton used this idea to formalize his first law of motion, forming "
            "the basis of classical mechanics."
        ),
        metadata={"domain": "physics", "law": "motion", "scientist": "Newton", "doc_id": 2}
    ),
    Document(
        page_content=(
            "In classical physics, Newton’s first law explains why objects in space continue moving "
            "in straight lines at constant speeds when no forces act on them."
        ),
        metadata={"domain": "physics", "law": "motion", "scientist": "Newton", "doc_id": 3}
    ),

    # ================= Law of Gravitation =================
    Document(
        page_content=(
            "The law of universal gravitation states that every mass attracts every other mass. "
            "The strength of this attraction depends on the masses involved and the distance between them."
        ),
        metadata={"domain": "physics", "law": "gravity", "scientist": "Newton", "doc_id": 4}
    ),
    Document(
        page_content=(
            "Newton’s theory of gravity explains planetary motion, falling objects, and tidal forces. "
            "It shows that gravity is a universal force acting across space."
        ),
        metadata={"domain": "physics", "law": "gravity", "scientist": "Newton", "doc_id": 5}
    ),
    Document(
        page_content=(
            "Gravitational attraction governs how planets orbit stars and how galaxies form. "
            "Newton mathematically described this force in his law of gravitation."
        ),
        metadata={"domain": "physics", "law": "gravity", "scientist": "Newton", "doc_id": 6}
    ),

    # ================= First Law of Thermodynamics =================
    Document(
        page_content=(
            "The first law of thermodynamics states that energy cannot be created or destroyed, "
            "only converted from one form to another."
        ),
        metadata={"domain": "physics", "law": "thermodynamics_1", "doc_id": 7}
    ),
    Document(
        page_content=(
            "Energy conservation is a fundamental principle expressed by the first law of thermodynamics. "
            "It applies to engines, chemical reactions, and biological systems."
        ),
        metadata={"domain": "physics", "law": "thermodynamics_1", "doc_id": 8}
    ),
    Document(
        page_content=(
            "According to thermodynamic principles, the total energy of an isolated system remains constant, "
            "even though energy may change form."
        ),
        metadata={"domain": "physics", "law": "thermodynamics_1", "doc_id": 9}
    ),

    # ================= Second Law of Thermodynamics =================
    Document(
        page_content=(
            "The second law of thermodynamics introduces entropy, stating that disorder in an isolated "
            "system tends to increase over time."
        ),
        metadata={"domain": "physics", "law": "thermodynamics_2", "doc_id": 10}
    ),
    Document(
        page_content=(
            "Entropy explains why heat flows naturally from hot objects to colder ones and not the reverse."
        ),
        metadata={"domain": "physics", "law": "thermodynamics_2", "doc_id": 11}
    ),
    Document(
        page_content=(
            "The irreversible nature of many natural processes can be explained using the second law "
            "of thermodynamics and the concept of entropy."
        ),
        metadata={"domain": "physics", "law": "thermodynamics_2", "doc_id": 12}
    ),

    # ================= Natural Selection =================
    Document(
        page_content=(
            "Natural selection explains how organisms with advantageous traits are more likely to survive "
            "and reproduce in a given environment."
        ),
        metadata={"domain": "biology", "law": "natural_selection", "scientist": "Darwin", "doc_id": 13}
    ),
    Document(
        page_content=(
            "Darwin’s theory of evolution states that species evolve gradually as beneficial traits "
            "become more common across generations."
        ),
        metadata={"domain": "biology", "law": "natural_selection", "scientist": "Darwin", "doc_id": 14}
    ),
    Document(
        page_content=(
            "Evolution by natural selection describes how environmental pressures shape populations "
            "over long periods of time."
        ),
        metadata={"domain": "biology", "law": "natural_selection", "scientist": "Darwin", "doc_id": 15}
    ),

    # ================= Cause and Effect =================
    Document(
        page_content=(
            "The law of cause and effect suggests that every event has a prior cause that leads to it."
        ),
        metadata={"domain": "philosophy", "law": "causality", "doc_id": 16}
    ),
    Document(
        page_content=(
            "Causality is a fundamental assumption in science, enabling prediction and explanation of natural phenomena."
        ),
        metadata={"domain": "philosophy", "law": "causality", "doc_id": 17}
    ),
    Document(
        page_content=(
            "Scientific reasoning relies on cause-and-effect relationships to understand how systems behave over time."
        ),
        metadata={"domain": "philosophy", "law": "causality", "doc_id": 18}
    ),
]


In [30]:
embedding_model = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2"
)

vector_store = FAISS.from_documents(
    documents,
    embedding_model
)

base_retriever = vector_store.as_retriever(search_kwargs={"k": 3})


In [32]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor

# create llm
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

generation_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=False
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

# compress the documents generated from the base_retriever
compressor = LLMChainExtractor.from_llm(llm)

# create a contextual retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
question ="Explain the law of gravity in simple terms"
result = compression_retriever.invoke(question)
for i, doc in enumerate(result):
  print(f"\n -- Result --{n+1}")
  print(doc.page_content)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
